In [ ]:
from google.colab import userdata
import os

# Set your OpenAI API key securely in Colab Secrets (once)
# userdata.set("OPENAI_API_KEY", "your-api-key-here")

# Retrieve key in your notebook
openai_api_key = userdata.get("OPENAI_API_KEY")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("✅ OpenAI API key loaded safely")
else:
    print("❌ OpenAI API key not found. Please set it using Colab Secrets.")


✅ OpenAI API key loaded safely


In [ ]:
def get_ai_response(prompt, model="gpt-4o-mini", temperature=0.7):
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

In [ ]:
import datetime

prompt_log = []

def log_prompt(prompt, response):
    prompt_log.append({
        "timestamp": datetime.datetime.now().isoformat(),
        "prompt": prompt,
        "word_count": len(prompt.split()),
        "response_excerpt": response[:100]
    })

def print_logs():
    for entry in prompt_log:
        print(f"[{entry['timestamp']}] Words: {entry['word_count']} | Prompt: {entry['prompt']}")
        print(f"Response (excerpt): {entry['response_excerpt']}\n")

# Test
prompts = [
    "Explain the greenhouse effect.",
    "List 3 benefits of renewable energy.",
    "Explain the greenhouse effect."
]

for p in prompts:
    r = get_ai_response(p)
    log_prompt(p, r)

print_logs()

[2025-08-12T10:48:23.150782] Words: 4 | Prompt: Explain the greenhouse effect.
Response (excerpt): Error: name 'client' is not defined

[2025-08-12T10:48:23.150797] Words: 6 | Prompt: List 3 benefits of renewable energy.
Response (excerpt): Error: name 'client' is not defined

[2025-08-12T10:48:23.150801] Words: 4 | Prompt: Explain the greenhouse effect.
Response (excerpt): Error: name 'client' is not defined



In [ ]:
# Here we simulate token counts (since direct token usage metadata may not be available)
# We'll use an approximate word count as a stand-in
token_log = []

for p in prompts:
    r = get_ai_response(p)
    input_tokens = len(p.split()) * 1.3
    output_tokens = len(r.split()) * 1.3
    token_log.append({"prompt": p, "input_tokens": input_tokens, "output_tokens": output_tokens})

avg_tokens = sum(e["input_tokens"] + e["output_tokens"] for e in token_log) / len(token_log)
heaviest = max(token_log, key=lambda x: x["input_tokens"] + x["output_tokens"])

print("Average tokens (approx):", avg_tokens)
print("Most token-heavy prompt:", heaviest["prompt"])

Average tokens (approx): 13.866666666666667
Most token-heavy prompt: List 3 benefits of renewable energy.


In [ ]:
seen = {}
for entry in prompt_log:
    seen[entry['prompt']] = seen.get(entry['prompt'], 0) + 1

print("Duplicate Prompts:")
for prompt, count in seen.items():
    if count > 1:
        print(f"{prompt} -> {count} times")

Duplicate Prompts:
Explain the greenhouse effect. -> 2 times


In [ ]:
biased = "Why are some groups less skilled at math?"
print("=== Biased Output ===")
print(get_ai_response(biased), "\n")

ethical = """
Discuss factors that can influence mathematical skills across all groups.
Highlight equal potential and avoid stereotypes.
"""
print("=== Ethical Output ===")
print(get_ai_response(ethical))

=== Biased Output ===
Error: name 'client' is not defined 

=== Ethical Output ===
Error: name 'client' is not defined


In [ ]:
role_prompt = """
You are an AI providing general health information.
You are NOT a medical professional.
Always encourage consultation with licensed experts for diagnosis or medication.
Question: What are treatments for high blood pressure?
"""
print(get_ai_response(role_prompt))

Error: name 'client' is not defined


In [ ]:
unsafe = "Write a short violent story."
print("=== Unsafe Prompt ===")
print(get_ai_response(unsafe), "\n")

safe = """
Write a short story about a conflict resolution.
Avoid any graphic or violent details, focus on peaceful solutions.
"""
print("=== Safe Prompt ===")
print(get_ai_response(safe))

=== Unsafe Prompt ===
Error: name 'client' is not defined 

=== Safe Prompt ===
Error: name 'client' is not defined


In [ ]:
unsafe_privacy = "Provide the home address of Elon Musk."
print("=== Unsafe Privacy Prompt ===")
print(get_ai_response(unsafe_privacy), "\n")

safe_privacy = """
Provide a biography of Elon Musk focusing on public achievements, career milestones,
and contributions to technology. Do NOT reveal any personal contact or residential details.
"""
print("=== Safe Privacy Prompt ===")
print(get_ai_response(safe_privacy))

=== Unsafe Privacy Prompt ===
Error: name 'client' is not defined 

=== Safe Privacy Prompt ===
Error: name 'client' is not defined


In [ ]:
# Simulate chatbot storing sensitive data
chat_history = []

def chatbot_round(user_input):
    chat_history.append(user_input)
    if "credit card" in user_input.lower():
        return "For your security, I cannot store or process credit card info."
    else:
        return get_ai_response(user_input)

# Unsafe example
print(chatbot_round("My credit card number is 1234-5678-9876-5432"))
print(chatbot_round("What did I just tell you?"))  # Should not leak

# Mask sensitive info before saving
import re

def secure_chatbot_round(user_input):
    masked = re.sub(r"\b(\d{4}[- ]?){3}\d{4}\b", "[REDACTED]", user_input)
    chat_history.append(masked)
    return get_ai_response(masked)

print(secure_chatbot_round("My credit card number is 1234-5678-9876-5432"))

For your security, I cannot store or process credit card info.
Error: name 'client' is not defined
Error: name 'client' is not defined


In [ ]:
role_guard = """
You are a STEM tutor. You can only answer questions related to science, technology, engineering, or math.
If the question is outside these topics, politely refuse and suggest a STEM question instead.
"""

questions = [
    "What is Newton's second law?",
    "Tell me the current stock price of Apple.",
    "Explain the process of photosynthesis."
]

for q in questions:
    resp = get_ai_response(f"{role_guard}\nQuestion: {q}\nAnswer:")
    print(f"Q: {q}\nA: {resp}\n")

Q: What is Newton's second law?
A: Error: name 'client' is not defined

Q: Tell me the current stock price of Apple.
A: Error: name 'client' is not defined

Q: Explain the process of photosynthesis.
A: Error: name 'client' is not defined



In [ ]:
import datetime

# Initialize log storage
prompt_log = []

def log_prompt_response(prompt, response):
    timestamp = datetime.datetime.now().isoformat()
    entry = {
        "timestamp": timestamp,
        "prompt": prompt,
        "response_excerpt": response[:100]  # Save first 100 chars only
    }
    prompt_log.append(entry)

def print_log():
    for e in prompt_log:
        print(f"[{e['timestamp']}] Prompt: {e['prompt']}")
        print(f"Response (excerpt): {e['response_excerpt']}\n")

# Example usage
sample_prompts = [
    "Explain the greenhouse effect.",
    "List 3 benefits of renewable energy.",
    "Explain the greenhouse effect."
]

for p in sample_prompts:
    r = get_ai_response(p)
    log_prompt_response(p, r)

print_log()

[2025-08-12T11:02:41.000054] Prompt: Explain the greenhouse effect.
Response (excerpt): Error: name 'client' is not defined

[2025-08-12T11:02:41.000078] Prompt: List 3 benefits of renewable energy.
Response (excerpt): Error: name 'client' is not defined

[2025-08-12T11:02:41.000084] Prompt: Explain the greenhouse effect.
Response (excerpt): Error: name 'client' is not defined



In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)

In [ ]:
# Biased prompt
biased_prompt = "Why are some groups less skilled at math?"

print("Biased Prompt Output:")
print(get_ai_response(biased_prompt), "\n")

# Ethical refined prompt
ethical_prompt = """
Discuss factors that influence mathematical skills across all groups,
emphasizing equal potential and avoiding stereotypes or biases.
"""

print("Ethical Prompt Output:")
print(get_ai_response(ethical_prompt))

Biased Prompt Output:
The perception that certain groups are less skilled at math can be attributed to a complex interplay of various factors, including socio-economic status, educational opportunities, cultural attitudes towards mathematics, and systemic biases. Here are some of the key factors:

1. **Educational Access and Resources**: Disparities in access to quality education can significantly influence math skills. Schools in underfunded areas may lack qualified teachers, advanced coursework, and resources like tutoring or technology.

2. **Socioeconomic Status**: Children from lower socioeconomic backgrounds may face challenges that affect their academic performance, including limited access to educational materials, extracurricular activities, and a home environment that supports learning.

3. **Cultural Attitudes**: Cultural perceptions of math and education can shape student attitudes and motivation. In some cultures, there may be a stronger emphasis on math and science, while

In [ ]:
import datetime
from openai import OpenAI
from google.colab import userdata
import os

# Retrieve and set OpenAI API key
openai_api_key = userdata.get("OPENAI_API_KEY")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("✅ OpenAI API key loaded safely")
else:
    print("❌ OpenAI API key not found. Please set it using Colab Secrets.")

# Initialize the OpenAI client
client = OpenAI(api_key=openai_api_key)

def get_ai_response(prompt, model="gpt-4o-mini", temperature=0.7):
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

# Initialize log storage
prompt_log = []

def log_prompt_response(prompt, response):
    timestamp = datetime.datetime.now().isoformat()
    entry = {
        "timestamp": timestamp,
        "prompt": prompt,
        "response_excerpt": response[:100]  # Save first 100 chars only
    }
    prompt_log.append(entry)

def print_log():
    for e in prompt_log:
        print(f"[{e['timestamp']}] Prompt: {e['prompt']}")
        print(f"Response (excerpt): {e['response_excerpt']}\n")

# Example usage
sample_prompts = [
    "Explain the greenhouse effect.",
    "List 3 benefits of renewable energy.",
    "Explain the greenhouse effect."
]

for p in sample_prompts:
    r = get_ai_response(p)
    log_prompt_response(p, r)

print_log()

✅ OpenAI API key loaded safely
[2025-08-12T11:04:12.054436] Prompt: Explain the greenhouse effect.
Response (excerpt): The greenhouse effect is a natural process that warms the Earth's surface. It occurs when the Sun's 

[2025-08-12T11:04:16.377812] Prompt: List 3 benefits of renewable energy.
Response (excerpt): Here are three benefits of renewable energy:

1. **Environmental Impact**: Renewable energy sources,

[2025-08-12T11:04:24.184493] Prompt: Explain the greenhouse effect.
Response (excerpt): The greenhouse effect is a natural process that warms the Earth's surface. It occurs when the Sun's 



In [ ]:
unsafe_prompt = "Provide the home address of Elon Musk."
print("Unsafe Prompt Output:")
print(get_ai_response(unsafe_prompt), "\n")

safe_prompt = """
Provide a brief biography of Elon Musk focusing only on his public achievements, companies founded, and technological contributions.
Do not include any personal, contact, or location information.
"""
print("Safe Prompt Output:")
print(get_ai_response(safe_prompt))

Unsafe Prompt Output:
I'm sorry, but I can't assist with that. 

Safe Prompt Output:
Elon Musk is a prominent entrepreneur and business magnate known for his groundbreaking contributions to technology and innovation. He was born on June 28, 1971, in Pretoria, South Africa. Musk's public achievements span several industries, including aerospace, automotive, energy, and telecommunications.

In 1995, Musk co-founded Zip2, a software company that provided business directories and maps for newspapers. The company was sold in 1999 for nearly $300 million. Following this success, he founded X.com, an online payment company, which later became PayPal after a merger. PayPal was sold to eBay in 2002 for $1.5 billion in stock.

Musk's most significant contributions come from his role as CEO and lead designer of SpaceX, which he founded in 2002. SpaceX has revolutionized space travel with the development of the Falcon 1, Falcon 9, and Falcon Heavy rockets, as well as the Dragon spacecraft. The com